In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from fastai import *
from fastai.datasets import *
from fastai.text import *
import pandas as pd
from tqdm import tqdm

# Data

In [0]:
path = Path('/content/drive/My Drive/Archieve/ValueLabs'); path.ls()

In [0]:
train_df = pd.read_csv(path/'Train.csv'); train_df.head()

In [0]:
bs = 24

# LM

In [0]:
data_lm = (TextList
           .from_df(train_df,cols=['question', 'answer_text', 'distractor'])
           .split_by_rand_pct(0.1)
           .label_for_lm()
           .databunch(bs=bs)
           )

In [0]:
data_lm.save(path/'lm.pkl')

In [0]:
data_lm = load_data(path, 'lm.pkl', bs=bs)
data_lm.show_batch()

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot(skip_end=10)

In [0]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [0]:
learn.save(path/'fit_head')

In [0]:
learn.load(path/'fit_head')

In [0]:
#@title Default title text
variable_name = "hello" #@param {type:"string"}

In [0]:
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7))

In [0]:
learn.save(path/'fine_tuned')

In [0]:
learn.load(path/'fine_tuned')

In [0]:
learn.predict("hi what is the problem",20)

In [0]:
test_df = pd.read_csv(path/'Test.csv');test_df.head()

In [0]:
combined = test_df['question']+test_df['answer_text']

In [0]:
combined.head()
combined.shape

In [0]:
output = []
for index, value in combined.iteritems():
    if index % 100 == 0: print(index)
    output.append(learn.predict(value))

In [0]:
import pickle

In [0]:
with open(path/'output_list','w') as f:
    pickle.dumps(output, f)

In [0]:
output[:5]